In [2]:
"""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import pandas as pd
#from sqlalchemy import create_engine

# Create list of dictionaries for each row.
listings = []
scrapedir = '/Users/rayd/workspace/cinema/scrapes/'
scrapefile = 'fandango-2019-05-16.csv'
scrape_path = scrapedir + scrapefile
with open(scrape_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   

# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
df = df[df.city != "Belleview, FL"]
df = df[df.city != "High Springs, FL"]
df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
times = (df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
times = times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
df = df.drop('times', axis=1)
df = pd.concat([df, times], axis=1, join_axes=[df.index])
df['streetaddy'] = df.addy.str.split(',').str[0]
df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df['cityaddy'] = df.addy.str.split(',').str[1]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('addy', axis=1)

# Ocala market
oc6_df = df.loc[df['theater'] == 'Ocala Center 6']
ocd_dr = df.loc[df['theater'] == 'Ocala Drive-In']
bv_df = df.loc[df['theater'] == 'Belleview Cinemas']
mt_df = df.loc[df['theater'] == 'Marion Theatre']
omp_df = df.loc[df['theater'] == 'Old Mill Playhouse']
regaloca_df = df.loc[df['theater'] == 'Regal Hollywood Stadium 16 & IMAX - Ocala']
rialto_df = df.loc[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']
amc_df = df.loc[df['theater'] == 'AMC Lake Square 12']
barn_df = df.loc[df['theater'] == 'Barnstorm Theater']

# Gainesville market
celeb_df = df.loc[df['theater'] == 'Celebration Pointe 10']
hipp_df = df.loc[df['theater'] == 'Hippodrome State Theatre']
butler_df = df.loc[df['theater'] == 'Regal Butler Town Center 14']
regalgnv_df = df.loc[df['theater'] == 'Regal Royal Park Stadium 16']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']
starke_df = df.loc[df['theater'] == 'Florida Twin'


In [9]:
oc6_df

,date,format,mpaa,stars,synopsis,theater,title,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,streetaddy,movieday,cityaddy
22,2019-05-16,Standard,PG-13,5,"Adrift in space with no food or water, Tony St...",Ocala Center 6,Avengers: Endgame (2019),12 p.m.,1 p.m.,3:45 p.m.,4:45 p.m.,7:30 p.m.,None,None,None,2021 Silver Springs Blvd.,Thursday,Ocala
37,2019-05-16,Standard,R,4,When Fred Flarsky (Seth Rogen) reunites with h...,Ocala Center 6,Long Shot,1 p.m.,4 p.m.,7 p.m.,None,None,None,None,None,2021 Silver Springs Blvd.,Thursday,Ocala
45,2019-05-16,Standard,PG,4.5,The story begins when ace detective Harry Good...,Ocala Center 6,Pokémon Detective Pikachu,1 p.m.,4 p.m.,7 p.m.,None,None,None,None,None,2021 Silver Springs Blvd.,Thursday,Ocala
117,2019-05-16,Standard,PG,4,"Unconventionality rules in UGLYDOLLS, the new ...",Ocala Center 6,UglyDolls,1 p.m.,4 p.m.,7 p.m.,None,None,None,None,None,2021 Silver Springs Blvd.,Thursday,Ocala
149,2019-05-16,Standard,PG-13,4,A psychological thriller about a young married...,Ocala Center 6,The Intruder (2019),1 p.m.,4 p.m.,7 p.m.,None,None,None,None,None,2021 Silver Springs Blvd.,Thursday,Ocala


In [1]:
import csv
import datetime
from datetime import date
# import glob
#from itertools import chain
import os.path, time
import numpy as np
import pandas as pd
#import shutil
#from tempfile import NamedTemporaryFile

newfile = "scrapes/fandango-2019-05-16.csv"

with open(newfile) as csv_file:
    reader = csv.reader(csv_file, delimiter= ',', quotechar='"')

    for row in reader:
        row[0] = row[0].split(', FL')[0]
        row[5] = row[5].replace('p | ', ' p.m., ')
        row[5] = row[5].replace('a | ', ' a.m., ')
        row[5] = row[5].replace('0p', '0 p.m.')
        row[5] = row[5].replace('5p', '5 p.m.')
        row[5] = row[5].replace(':00', '')
        row[5] = row[5].replace(' p.m.,', ',')

ocala_market = ['Ocala, FL', 'Belleview, FL', 'The Villages, FL']
gnv_market = ['Gainesville FL', 'High Springs, FL', 'Starke, FL']

amc_lake = 'AmC Lake Square 12'
barnstorm = 'Barnstorm Theater'
celebration10 = 'Celebration Point 10'
fla_twin = 'Florida Twin'
hipp = 'Hippodrome State Theatre'
marion_th = 'Marion Theatre'
ocala6 = 'Ocala Center 6'
old_mill = 'Old Mill Playhouse'
butler = 'Regal Butler Town Center 14'
imax_ocala = 'Regal Hollywood Stadium 16 & IMAX - Ocala'
regal_royal = 'Regal Royal Park Stadium 16'
reitz = 'Reitz Union Cinema - U of Fl'
rialto = 'Rialto Theatre Spanish Springs Town Square'

def gnvdaily_pn():
    """ Builds print narrative for Gainesville daily editions """
    listing = []
    listings = []
    ocala_market = ['Ocala, FL', 'Belleview, FL', 'The Villages, FL']
    gnv_market = ['Gainesville FL', 'High Springs, FL', 'Starke, FL']
    with open("scrapes/fandango-2019-05-16.csv") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        headers = next(csv_reader)
        for row in csv_reader:
            if row[9] in ocala_market:
                row[0] = row[0].split(', FL')[0]
                row[5] = row[5].replace('p | ', ' p.m., ')
                row[5] = row[5].replace('a | ', ' a.m., ')
                row[5] = row[5].replace('0p', '0 p.m.')
                row[5] = row[5].replace('5p', '5 p.m.')
                row[5] = row[5].replace(':00', '')
                movie_stuff = f"\"{row[2]}\", {row[3]}, {row[4]}, {row[6]}"
                theater_stuff = f'{row[1]}, {row[0]}, {row[5]}'
                listing = movie_stuff + theater_stuff
                listings.append(listing)
    return listings

gnvdaily_pn()


['"Breakthrough (2019)", PG, 4.5, StandardAMC Lake Square 12, 10401-015 US Hwy. 441 South, Leesburg, 10:25 a.m., 1:10 p.m.',
 '"Saga of Tanya the Evil – the Movie", , , StandardRegal Hollywood & IMAX - Ocala, 2801 SW 27th Avenue, Ocala, 7:30 p.m.',
 '"Avengers: Endgame (2019)", PG-13, 5, StandardBelleview Cinemas, 10845 SE US Highway 441, Belleview, 1 p.m., 4:30 p.m., 8 p.m.',
 '"The Hustle (2019)", PG-13, 3.5, StandardRegal Hollywood & IMAX - Ocala, 2801 SW 27th Avenue, Ocala, 2:25 p.m., 5 p.m., 7:35 p.m., 10:10 p.m.',
 '"", , , Ocala Drive-In, 4850 S. Pine Ave., Ocala, ',
 '"Long Shot", R, 4, StandardRegal Hollywood & IMAX - Ocala, 2801 SW 27th Avenue, Ocala, 7:55 p.m., 10:50 p.m.',
 '"Mia and the White Lion", PG, 4.5, StandardRialto Theatre Spanish Springs Town Square, 1105 Alonzo Ave, Lady Lake, 9:30 a.m., 2:05 p.m.',
 '"Unplanned (2019)", R, 4.5, StandardOld Mill Playhouse, 1000 Old Mill Run, The Villages, 10 a.m., 12:30 p.m., 3 p.m., 5:30 p.m., 8 p.m.',
 '"Pokémon Detective Pikac